<a href="https://colab.research.google.com/github/XIAOBAOWEI158788/GeminiProChat/blob/main/%E2%80%9Cquick_demo_ipynb%E2%80%9D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## VideoReTalking：Audio-based Lip Synchronization for Talking Head Video Editing In the Wild

[Arxiv](https://arxiv.org/abs/2211.14758) | [Project](https://vinthony.github.io/video-retalking/) | [Github](https://github.com/vinthony/video-retalking)

Kun Cheng, Xiaodong Cun, Yong Zhang, Menghan Xia, Fei Yin, Mingrui Zhu, Xuan Wang, Jue Wang, Nannan Wang

Xidian University, Tencent AI Lab, Tsinghua University

*SIGGRAPH Asia 2022 Conferenence Track*



**Installation** (30s)

In [1]:
#@title
### make sure that CUDA is available in Edit -> Nootbook settings -> GPU
!nvidia-smi

!python --version
!apt-get update
!apt install ffmpeg &> /dev/null

print('Git clone project and install requirements...')
!git clone https://github.com/vinthony/video-retalking.git &> /dev/null
%cd video-retalking
# !pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html
!pip install -r requirements.txt

Sun Jun  2 07:38:08 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8              12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

**Download Pretrained Models**

In [2]:
#@title
print('Download pre-trained models...')
!mkdir ./checkpoints
!wget https://github.com/vinthony/video-retalking/releases/download/v0.0.1/30_net_gen.pth -O ./checkpoints/30_net_gen.pth
!wget https://github.com/vinthony/video-retalking/releases/download/v0.0.1/BFM.zip -O ./checkpoints/BFM.zip
!wget https://github.com/vinthony/video-retalking/releases/download/v0.0.1/DNet.pt -O ./checkpoints/DNet.pt
!wget https://github.com/vinthony/video-retalking/releases/download/v0.0.1/ENet.pth -O ./checkpoints/ENet.pth
!wget https://github.com/vinthony/video-retalking/releases/download/v0.0.1/expression.mat -O ./checkpoints/expression.mat
!wget https://github.com/vinthony/video-retalking/releases/download/v0.0.1/face3d_pretrain_epoch_20.pth -O ./checkpoints/face3d_pretrain_epoch_20.pth
!wget https://github.com/vinthony/video-retalking/releases/download/v0.0.1/GFPGANv1.3.pth -O ./checkpoints/GFPGANv1.3.pth
!wget https://github.com/vinthony/video-retalking/releases/download/v0.0.1/GPEN-BFR-512.pth -O ./checkpoints/GPEN-BFR-512.pth
!wget https://github.com/vinthony/video-retalking/releases/download/v0.0.1/LNet.pth -O ./checkpoints/LNet.pth
!wget https://github.com/vinthony/video-retalking/releases/download/v0.0.1/ParseNet-latest.pth -O ./checkpoints/ParseNet-latest.pth
!wget https://github.com/vinthony/video-retalking/releases/download/v0.0.1/RetinaFace-R50.pth -O ./checkpoints/RetinaFace-R50.pth
!wget https://github.com/vinthony/video-retalking/releases/download/v0.0.1/shape_predictor_68_face_landmarks.dat -O ./checkpoints/shape_predictor_68_face_landmarks.dat
!unzip -d ./checkpoints/BFM ./checkpoints/BFM.zip

Download pre-trained models...
--2024-06-02 07:38:30--  https://github.com/vinthony/video-retalking/releases/download/v0.0.1/30_net_gen.pth
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/OpenTalker/video-retalking/releases/download/v0.0.1/30_net_gen.pth [following]
--2024-06-02 07:38:31--  https://github.com/OpenTalker/video-retalking/releases/download/v0.0.1/30_net_gen.pth
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-production-release-asset-2e65be/536411820/38c02c2b-bf57-4d4e-9711-6cefdc5c817c?X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Credential=releaseassetproduction%2F20240602%2Fus-east-1%2Fs3%2Faws4_request&X-Amz-Date=20240602T073831Z&X-Amz-Expires=300&X-Amz-Signature=c5f60e694f61e5688d40ea8f2147b366c33780209d07ef

**Inference**

`--face`: Input video.

`--audio`: Input audio. Both *.wav* and *.mp4* files are supported.

You can choose our provided data from `./examples` folder or upload from your local computer.






In [ ]:
#@title
import glob, os, sys
import ipywidgets as widgets
from IPython.display import HTML
from base64 import b64encode

print("Choose the Video name to edit: (saved in folder 'examples/face')")
vid_list = glob.glob1('examples/face/', '*.mp4')
vid_list.sort()
default_vid_name = widgets.Dropdown(options=vid_list, value='1.mp4')
display(default_vid_name)

print("Choose the Audio name to edit: (saved in folder 'examples/audio')")
audio_list = glob.glob1('examples/audio/', '*')
audio_list.sort()
default_audio_name = widgets.Dropdown(options=audio_list, value='1.wav')
display(default_audio_name)


Visualize the input video and audio:

In [ ]:
#@title
input_video_name = './examples/face/{}'.format(default_vid_name.value)
input_video_mp4 = open('{}'.format(input_video_name),'rb').read()
input_video_data_url = "data:video/x-m4v;base64," + b64encode(input_video_mp4).decode()
print('Display input video: {}'.format(input_video_name), file=sys.stderr)
display(HTML("""
  <video width=400 controls>
        <source src="%s" type="video/mp4">
  </video>
  """ % input_video_data_url))

input_audio_name = './examples/audio/{}'.format(default_audio_name.value)
input_audio_mp4 = open('{}'.format(input_audio_name),'rb').read()
input_audio_data_url = "data:audio/wav;base64," + b64encode(input_audio_mp4).decode()
print('Display input audio: {}'.format(input_audio_name), file=sys.stderr)
display(HTML("""
  <audio width=400 controls>
        <source src="%s" type="audio/wav">
  </audio>
  """ % input_audio_data_url))


In [ ]:
input_video_path = 'examples/face/{}'.format(default_vid_name.value)
input_audio_path = 'examples/audio/{}'.format(default_audio_name.value)

!python3 inference.py \
  --face {input_video_path} \
  --audio {input_audio_path} \
  --outfile results/output.mp4

Visualize the output video:

In [ ]:
#@title
# visualize code from makeittalk
from IPython.display import HTML
from base64 import b64encode
import os, sys, glob, cv2, subprocess, platform

def read_video(vid_name):
  video_stream = cv2.VideoCapture(vid_name)
  fps = video_stream.get(cv2.CAP_PROP_FPS)
  full_frames = []
  while True:
    still_reading, frame = video_stream.read()
    if not still_reading:
        video_stream.release()
        break
    full_frames.append(frame)
  return full_frames, fps

input_video_frames, fps = read_video(input_video_path)
output_video_frames, _ = read_video('./results/output.mp4')

frame_h, frame_w = input_video_frames[0].shape[:-1]
out_concat = cv2.VideoWriter('./temp/temp/result_concat.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_w*2, frame_h))
for i in range(len(output_video_frames)):
  frame_input = input_video_frames[i % len(input_video_frames)]
  frame_output = output_video_frames[i]
  out_concat.write(cv2.hconcat([frame_input, frame_output]))
out_concat.release()

command = 'ffmpeg -loglevel error -y -i {} -i {} -strict -2 -q:v 1 {}'.format(input_audio_path, './temp/temp/result_concat.mp4', './results/output_concat_input.mp4')
subprocess.call(command, shell=platform.system() != 'Windows')


output_video_name = './results/output.mp4'
output_video_mp4 = open('{}'.format(output_video_name),'rb').read()
output_video_data_url = "data:video/mp4;base64," + b64encode(output_video_mp4).decode()
print('Display lip-syncing video: {}'.format(output_video_name), file=sys.stderr)
display(HTML("""
  <video height=400 controls>
        <source src="%s" type="video/mp4">
  </video>
  """ % output_video_data_url))

output_concat_video_name = './results/output_concat_input.mp4'
output_concat_video_mp4 = open('{}'.format(output_concat_video_name),'rb').read()
output_concat_video_data_url = "data:video/mp4;base64," + b64encode(output_concat_video_mp4).decode()
print('Display input video and lip-syncing video: {}'.format(output_concat_video_name), file=sys.stderr)
display(HTML("""
  <video height=400 controls>
        <source src="%s" type="video/mp4">
  </video>
  """ % output_concat_video_data_url))
